In [ ]:
# importing the pandas module for
# data frame
!pip install pygam
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from pygam import LinearGAM
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score




# load the data set into train variable.
train = pd.read_csv('/kaggle/input/trafficvehiclesjunction/traffic.csv')
train = train.drop('ID', axis=1)

# display top 5 values of data set
train.head()



^C
ERROR: Operation cancelled by user


In [ ]:
# function to get all data from time stamp

# get date
def get_dom(dt):
	return dt.day

# get week day
def get_weekday(dt):
	return dt.weekday()

# get hour
def get_hour(dt):
	return dt.hour

# get year
def get_year(dt):
	return dt.year

# get month
def get_month(dt):
	return dt.month

# get year day
def get_dayofyear(dt):
	return dt.dayofyear

# get year week
def get_weekofyear(dt):
	return dt.weekofyear


train['DateTime'] = train['DateTime'].map(pd.to_datetime)
train['date'] = train['DateTime'].map(get_dom)
train['weekday'] = train['DateTime'].map(get_weekday)
train['hour'] = train['DateTime'].map(get_hour)
train['month'] = train['DateTime'].map(get_month)
train['year'] = train['DateTime'].map(get_year)
train['dayofyear'] = train['DateTime'].map(get_dayofyear)
train['weekofyear'] = train['DateTime'].map(get_weekofyear)

# display
train.head()


In [ ]:
train.to_csv('file1.csv')

In [ ]:
# display top 5 values of data set
#train = train[train['Junction'] == 1]
train.head()

In [ ]:
#train = train.drop(['Junction'], axis=1)
print(train.columns)

In [ ]:
# there is no use of DateTime module
# so remove it
train = train.drop(['DateTime'], axis=1)

# separating class label for training the data
train1 = train.drop(['Vehicles'], axis=1)

# class label is stored in target
target = train['Vehicles']

print(train1.head())
target.head()


In [ ]:
target = pd.read_csv('/kaggle/working/file1.csv')

# separating class label for training the data
train1 = train.drop('Vehicles', axis=1)

# class label is stored in target
target = train['Vehicles']

train1.head()
#target.head()


In [ ]:

# train1 = pd.read_csv('/kaggle/working/train1.csv')
train1 = train1.fillna(train1.mean())
# target = pd.read_csv('/kaggle/working/target.csv')
#importing Random forest
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
#defining the RandomForestRegressor

# k = 3
# m1=RandomForestRegressor()
# from catboost import CatBoostRegressor
# m1 = CatBoostRegressor(loss_function='MAE',iterations=10000,verbose=False)

xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42,n_estimators=300)
catboost_model = CatBoostRegressor(loss_function='MAE',iterations=50000,verbose=False)
# Training the model
#m1.fit(train1, target)
# m1 = LinearGAM().fit(train1, target)

# m1.fit(train1,target)
#testing
# m1.predict([[11,6,0,1,2015,11,2]])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train1, target, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
# Create and train the model
# model = DecisionTreeClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
catboost_model.fit(X_train, y_train, verbose=False)
xgb_preds_val = xgb_model.predict(X_val)
catboost_preds_val = catboost_model.predict(X_val)
xgb_preds_test = xgb_model.predict(X_test)
catboost_preds_test = catboost_model.predict(X_test)
# Ensemble methods
# Simple Averaging
ensemble_preds_avg = (xgb_preds_test + catboost_preds_test) / 2

# Weighted Averaging (hypothetical weights, adjust according to performance)
ensemble_preds_weighted = (0.7 * xgb_preds_test) + (0.3 * catboost_preds_test)

# Stacking (use predictions of xgb_model and catboost_model as features)


In [20]:
import sklearn
stacking_X = np.column_stack((xgb_preds_val, catboost_preds_val))
meta_model = sklearn.linear_model.LinearRegression()
meta_model.fit(stacking_X, y_val)
stack_test=np.column_stack((xgb_preds_test,catboost_preds_test))
stacking_preds = meta_model.predict(stack_test)


# Evaluate the model
mae = mean_absolute_error(y_test, ensemble_preds_avg)
mse = mean_squared_error(y_test, ensemble_preds_avg)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, ensemble_preds_avg)

print(f'simple averaging')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

print(f'weighted averaging')
mae = mean_absolute_error(y_test, ensemble_preds_weighted)
mse = mean_squared_error(y_test, ensemble_preds_weighted)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, ensemble_preds_weighted)

print(f'stacking')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

mae = mean_absolute_error(y_test, stacking_preds)
mse = mean_squared_error(y_test, stacking_preds)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, stacking_preds)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

simple averaging
Mean Absolute Error: 2.2290340203499044
Mean Squared Error: 14.723035807633691
Root Mean Squared Error: 3.837060829285052
R-squared: 0.9632183003264417
weighted averaging
stacking
Mean Absolute Error: 2.257694826989963
Mean Squared Error: 14.159616027550491
Root Mean Squared Error: 3.762926524335878
R-squared: 0.9646258590264225
Mean Absolute Error: 2.346124974864848
Mean Squared Error: 14.211458212704597
Root Mean Squared Error: 3.7698087766761588
R-squared: 0.9644963447258615
